In [1]:
versioninfo()

Julia Version 1.5.1
Commit 697e782ab8 (2020-08-25 20:08 UTC)
Platform Info:
  OS: Linux (x86_64-pc-linux-gnu)
  CPU: Intel(R) Core(TM) i7-9750H CPU @ 2.60GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-9.0.1 (ORCJIT, skylake)


In [2]:
]instantiate

In [3]:
using Squareness

## Simple Newton Method

In [4]:
function _issquare_with_simple_newton(x::I) where {I <: Integer}
    x == 0 && return true
    xtz = trailing_zeros(x)
    isodd(xtz) && return false
    x >>= unsigned(xtz)
    o_xrt::I = x
    xrt::I = (x + 0x1) >> 0x1
    xrt == 0 && return false  # overflow
    while xrt < o_xrt
        o_xrt = xrt
        xrt = (xrt + x ÷ xrt) >> 0x1
    end
    xrt * xrt == x
end

issquare_with_simple_newton(x::Signed) = x < 0 ? false : _issquare_with_simple_newton(x % Unsigned)
issquare_with_simple_newton(x::Unsigned) = _issquare_with_simple_newton(x)
issquare_with_simple_newton(x::BigInt) = x < 0 ? false : _issquare_with_simple_newton(x)

issquare_with_simple_newton (generic function with 3 methods)

### Simple Check

In [5]:
all(issquare(x) == issquare_with_simple_newton(x) == Squareness.issquare_with_isqrt(x) for x in rand(Int, 10000))

true

In [6]:
all(issquare(x) == issquare_with_simple_newton(x) == Squareness.issquare_with_isqrt(x) for x in rand(Int32, 10000))

true

## Benchmark

In [7]:
using BenchmarkTools

### Compare `issquare()`(with P-adic Newton) vs `issquare_with_isqrt()` vs `issquare_with_simple_newton()`

#### `Int64`

In [8]:
rand_int64() = rand(Int64) & typemax(Int64)

rand_int64 (generic function with 1 method)

In [9]:
@benchmark issquare(n) setup=(n=rand_int64())

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     2.946 ns (0.00% GC)
  median time:      5.236 ns (0.00% GC)
  mean time:        4.796 ns (0.00% GC)
  maximum time:     276.003 ns (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     1000

In [10]:
@benchmark Squareness.issquare_with_isqrt(n) setup=(n=rand_int64())

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     12.608 ns (0.00% GC)
  median time:      12.723 ns (0.00% GC)
  mean time:        12.941 ns (0.00% GC)
  maximum time:     58.036 ns (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     999

In [11]:
@benchmark issquare_with_simple_newton(n) setup=(n=rand_int64())

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     2.402 ns (0.00% GC)
  median time:      330.067 ns (0.00% GC)
  mean time:        230.434 ns (0.00% GC)
  maximum time:     627.928 ns (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     1000

#### `Int64` × 100

In [12]:
rand_int64(N) = rand(Int64, N) .& typemax(Int64)

rand_int64 (generic function with 2 methods)

In [13]:
@benchmark issquare.(n) setup=(n=rand_int64(100))

BenchmarkTools.Trial: 
  memory estimate:  128 bytes
  allocs estimate:  2
  --------------
  minimum time:     495.487 ns (0.00% GC)
  median time:      587.832 ns (0.00% GC)
  mean time:        610.786 ns (1.18% GC)
  maximum time:     15.970 μs (96.39% GC)
  --------------
  samples:          10000
  evals/sample:     193

In [14]:
@benchmark Squareness.issquare_with_isqrt.(n) setup=(n=rand_int64(100))

BenchmarkTools.Trial: 
  memory estimate:  128 bytes
  allocs estimate:  2
  --------------
  minimum time:     1.452 μs (0.00% GC)
  median time:      1.536 μs (0.00% GC)
  mean time:        1.589 μs (0.00% GC)
  maximum time:     6.178 μs (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     10

In [15]:
@benchmark issquare_with_simple_newton.(n) setup=(n=rand_int64(100))

BenchmarkTools.Trial: 
  memory estimate:  128 bytes
  allocs estimate:  2
  --------------
  minimum time:     16.471 μs (0.00% GC)
  median time:      23.184 μs (0.00% GC)
  mean time:        23.704 μs (0.00% GC)
  maximum time:     83.584 μs (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     1

#### `Int32`

In [16]:
rand_int32() = rand(Int32) & typemax(Int32)

rand_int32 (generic function with 1 method)

In [17]:
@benchmark issquare(n) setup=(n=rand_int32())

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     2.653 ns (0.00% GC)
  median time:      4.302 ns (0.00% GC)
  mean time:        3.833 ns (0.00% GC)
  maximum time:     37.422 ns (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     1000

In [18]:
@benchmark Squareness.issquare_with_isqrt(n) setup=(n=rand_int32())

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     7.528 ns (0.00% GC)
  median time:      7.635 ns (0.00% GC)
  mean time:        7.776 ns (0.00% GC)
  maximum time:     40.907 ns (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     999

In [19]:
@benchmark issquare_with_simple_newton(n) setup=(n=rand_int32())

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     3.120 ns (0.00% GC)
  median time:      103.523 ns (0.00% GC)
  mean time:        74.506 ns (0.00% GC)
  maximum time:     236.730 ns (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     1000

#### `Int32` × 100

In [20]:
rand_int32(N) = rand(Int32, N) .& typemax(Int32)

rand_int32 (generic function with 2 methods)

In [21]:
@benchmark issquare.(n) setup=(n=rand_int32(100))

BenchmarkTools.Trial: 
  memory estimate:  128 bytes
  allocs estimate:  2
  --------------
  minimum time:     424.685 ns (0.00% GC)
  median time:      516.629 ns (0.00% GC)
  mean time:        540.323 ns (1.35% GC)
  maximum time:     15.378 μs (96.61% GC)
  --------------
  samples:          10000
  evals/sample:     197

In [22]:
@benchmark Squareness.issquare_with_isqrt.(n) setup=(n=rand_int32(100))

BenchmarkTools.Trial: 
  memory estimate:  128 bytes
  allocs estimate:  2
  --------------
  minimum time:     392.687 ns (0.00% GC)
  median time:      417.348 ns (0.00% GC)
  mean time:        431.163 ns (1.69% GC)
  maximum time:     15.481 μs (97.27% GC)
  --------------
  samples:          10000
  evals/sample:     201

In [23]:
@benchmark issquare_with_simple_newton.(n) setup=(n=rand_int32(100))

BenchmarkTools.Trial: 
  memory estimate:  128 bytes
  allocs estimate:  2
  --------------
  minimum time:     5.753 μs (0.00% GC)
  median time:      8.066 μs (0.00% GC)
  mean time:        8.212 μs (0.00% GC)
  maximum time:     21.695 μs (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     6

#### `Int16`

In [24]:
rand_int16() = rand(Int16) & typemax(Int16)

rand_int16 (generic function with 1 method)

In [25]:
@benchmark issquare(n) setup=(n=rand_int16())

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     2.876 ns (0.00% GC)
  median time:      9.583 ns (0.00% GC)
  mean time:        7.569 ns (0.00% GC)
  maximum time:     54.919 ns (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     1000

In [26]:
@benchmark Squareness.issquare_with_isqrt(n) setup=(n=rand_int16())

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     6.624 ns (0.00% GC)
  median time:      7.876 ns (0.00% GC)
  mean time:        8.085 ns (0.00% GC)
  maximum time:     28.365 ns (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     999

In [27]:
@benchmark issquare_with_simple_newton(n) setup=(n=rand_int16())

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     2.713 ns (0.00% GC)
  median time:      54.332 ns (0.00% GC)
  mean time:        41.261 ns (0.00% GC)
  maximum time:     143.355 ns (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     1000

#### `Int16` × 100

In [28]:
rand_int16(N) = rand(Int16, N) .& typemax(Int16)

rand_int16 (generic function with 2 methods)

In [29]:
@benchmark issquare.(n) setup=(n=rand_int16(100))

BenchmarkTools.Trial: 
  memory estimate:  128 bytes
  allocs estimate:  2
  --------------
  minimum time:     405.844 ns (0.00% GC)
  median time:      493.882 ns (0.00% GC)
  mean time:        531.313 ns (1.65% GC)
  maximum time:     20.764 μs (96.89% GC)
  --------------
  samples:          10000
  evals/sample:     199

In [30]:
@benchmark Squareness.issquare_with_isqrt.(n) setup=(n=rand_int16(100))

BenchmarkTools.Trial: 
  memory estimate:  128 bytes
  allocs estimate:  2
  --------------
  minimum time:     410.350 ns (0.00% GC)
  median time:      445.535 ns (0.00% GC)
  mean time:        498.712 ns (2.11% GC)
  maximum time:     33.145 μs (97.66% GC)
  --------------
  samples:          10000
  evals/sample:     200

In [31]:
@benchmark issquare_with_simple_newton.(n) setup=(n=rand_int16(100))

BenchmarkTools.Trial: 
  memory estimate:  128 bytes
  allocs estimate:  2
  --------------
  minimum time:     2.550 μs (0.00% GC)
  median time:      3.461 μs (0.00% GC)
  mean time:        3.555 μs (0.00% GC)
  maximum time:     10.481 μs (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     9

#### `Int128`

In [32]:
rand_int128() = rand(Int128) & typemax(Int128)

rand_int128 (generic function with 1 method)

In [33]:
@benchmark issquare(n) setup=(n=rand_int128())

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     2.710 ns (0.00% GC)
  median time:      26.044 ns (0.00% GC)
  mean time:        18.837 ns (0.00% GC)
  maximum time:     73.259 ns (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     1000

In [34]:
@benchmark Squareness.issquare_with_isqrt(n) setup=(n=rand_int128())

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     45.116 ns (0.00% GC)
  median time:      45.558 ns (0.00% GC)
  mean time:        46.289 ns (0.00% GC)
  maximum time:     116.919 ns (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     989

In [35]:
@benchmark issquare_with_simple_newton(n) setup=(n=rand_int128())

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     3.121 ns (0.00% GC)
  median time:      1.842 μs (0.00% GC)
  mean time:        1.264 μs (0.00% GC)
  maximum time:     2.501 μs (0.00% GC)
  --------------
  samples:          3957
  evals/sample:     1000

#### `Int128` × 100

In [36]:
rand_int128(N) = rand(Int128, N) .& typemax(Int128)

rand_int128 (generic function with 2 methods)

In [37]:
@benchmark issquare.(n) setup=(n=rand_int128(100))

BenchmarkTools.Trial: 
  memory estimate:  128 bytes
  allocs estimate:  2
  --------------
  minimum time:     1.524 μs (0.00% GC)
  median time:      1.977 μs (0.00% GC)
  mean time:        2.036 μs (0.83% GC)
  maximum time:     170.484 μs (98.67% GC)
  --------------
  samples:          10000
  evals/sample:     10

In [38]:
@benchmark Squareness.issquare_with_isqrt.(n) setup=(n=rand_int128(100))

BenchmarkTools.Trial: 
  memory estimate:  128 bytes
  allocs estimate:  2
  --------------
  minimum time:     4.557 μs (0.00% GC)
  median time:      4.618 μs (0.00% GC)
  mean time:        4.762 μs (0.00% GC)
  maximum time:     13.873 μs (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     7

In [39]:
@benchmark issquare_with_simple_newton.(n) setup=(n=rand_int128(100))

BenchmarkTools.Trial: 
  memory estimate:  128 bytes
  allocs estimate:  2
  --------------
  minimum time:     90.002 μs (0.00% GC)
  median time:      129.335 μs (0.00% GC)
  mean time:        131.035 μs (0.00% GC)
  maximum time:     320.768 μs (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     1

#### `BigInt`

In [40]:
rand_mpz() = (big(rand(UInt128)) << rand(0:128)) + rand(UInt128)

rand_mpz (generic function with 1 method)

In [41]:
# @benchmark issquare(n) setup=(n=rand_mpz())
@benchmark Squareness.issquare_with_padic_newton(n) setup=(n=rand_mpz())

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     7.180 ns (0.00% GC)
  median time:      300.697 μs (1.26% GC)
  mean time:        262.128 μs (3.04% GC)
  maximum time:     567.948 μs (1.24% GC)
  --------------
  samples:          20
  evals/sample:     999

In [42]:
@benchmark Squareness.issquare_with_isqrt(n) setup=(n=rand_mpz())

BenchmarkTools.Trial: 
  memory estimate:  88 bytes
  allocs estimate:  5
  --------------
  minimum time:     133.171 ns (0.00% GC)
  median time:      234.856 ns (0.00% GC)
  mean time:        384.628 ns (20.68% GC)
  maximum time:     123.977 μs (76.68% GC)
  --------------
  samples:          10000
  evals/sample:     864

In [43]:
@benchmark issquare_with_simple_newton(n) setup=(n=rand_mpz())

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     8.371 ns (0.00% GC)
  median time:      21.862 μs (0.00% GC)
  mean time:        30.818 μs (24.43% GC)
  maximum time:     135.672 μs (56.86% GC)
  --------------
  samples:          163
  evals/sample:     999